<a href="https://colab.research.google.com/github/preetamjumech/CNN_from_Scratch/blob/main/Preetam_Saha_CNN_MNIST_practice_24_09_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/test.pt

--2022-09-24 09:22:21--  https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt [following]
--2022-09-24 09:22:21--  https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47520431 (45M) [application/octet-stream]
Saving to: ‘training.pt.1’

training.pt.1       100%[===================

In [2]:
import torch, torchsummary
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, RMSprop
import numpy as np

In [3]:
(x_train,y_train) , (x_test,y_test) = torch.load("training.pt"), torch.load("test.pt")

In [4]:
x_train.shape , y_train.shape

(torch.Size([60000, 28, 28]), torch.Size([60000]))

In [5]:
x_train = x_train.reshape(-1,1,28,28)
x_test = x_test.reshape(-1,1,28,28)

In [6]:
x_train.shape

torch.Size([60000, 1, 28, 28])

In [7]:
from PIL import Image

In [8]:
idx = 3
Image.fromarray(x_train[idx,0,:,:].numpy())

In [9]:
idx=200
Image.fromarray(x_train[idx,0,:,:].numpy())

In [10]:
idx=300
Image.fromarray(x_train[idx,0,:,:].numpy())

In [11]:
class MNIST_CNN(nn.Module):
  def __init__(self):
    super(MNIST_CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=16, kernel_size=3, padding=1 ) #in_channel = channel, out_channels=no of filters, kernel_Size = filter size
    self.conv2 = nn.Conv2d(in_channels=16,out_channels=32, kernel_size=3, padding=1 )
    self.conv3 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=3, padding=1 )
    self.conv4 = nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3, padding=1 )
    self.bn1 = nn.BatchNorm2d(num_features=16)
    self.bn2 = nn.BatchNorm2d(num_features=32)
    self.bn3 = nn.BatchNorm2d(num_features=64)
    self.pool = nn.MaxPool2d(kernel_size=2, stride = 2) #by pooling only reduction happenes, stride = 2 means jump of 2 pixels
    self.h1 = nn.Linear(in_features=1*1*128, out_features= 512)
    self.h2 = nn.Linear(in_features=512, out_features= 256)
    self.h3 = nn.Linear(in_features= 256, out_features= 32)
    self.out = nn.Linear(in_features = 32, out_features=10)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)

    x = self.bn1(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)

    x = self.bn2(x)
    x = F.relu(self.conv3(x))
    x = self.pool(x)

    x = self.bn3(x)
    x = F.relu(self.conv4(x))
    x = self.pool(x)
    #print(x.size())

    x = x.view(x.size()[0],-1)
    #print(x.size())

    x = F.relu(self.h1(x))
    x = F.relu(self.h2(x))
    x = F.relu(self.h3(x))
    x = self.out(x)

    return x

In [12]:
cnn_model = MNIST_CNN()

In [13]:
list(cnn_model.parameters())

[Parameter containing:
 tensor([[[[ 0.1611, -0.2510,  0.0426],
           [ 0.2098,  0.2426, -0.2421],
           [-0.1161, -0.1590, -0.0639]]],
 
 
         [[[ 0.2271, -0.0780, -0.2621],
           [-0.0097,  0.0516,  0.1884],
           [ 0.0849, -0.2967, -0.2091]]],
 
 
         [[[ 0.0325,  0.3007, -0.1510],
           [-0.2614,  0.1871, -0.0744],
           [ 0.2829, -0.2083,  0.3055]]],
 
 
         [[[-0.2672, -0.2952,  0.0706],
           [ 0.1891, -0.1290, -0.1860],
           [-0.1478,  0.3087,  0.0038]]],
 
 
         [[[ 0.2971,  0.2812,  0.1297],
           [ 0.0264, -0.0066, -0.1679],
           [-0.0712,  0.3110,  0.0675]]],
 
 
         [[[ 0.1692, -0.0034, -0.3161],
           [ 0.1067,  0.0094, -0.0882],
           [-0.1725, -0.0043, -0.1951]]],
 
 
         [[[-0.1159, -0.0849, -0.1509],
           [-0.0297,  0.1176,  0.1689],
           [ 0.2235, -0.1152, -0.0324]]],
 
 
         [[[ 0.1271,  0.1267, -0.2997],
           [ 0.2824, -0.0836, -0.3123],
           [ 0.

In [14]:
torchsummary.summary(cnn_model,input_size=(1,28,28),device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
         MaxPool2d-2           [-1, 16, 14, 14]               0
       BatchNorm2d-3           [-1, 16, 14, 14]              32
            Conv2d-4           [-1, 32, 14, 14]           4,640
         MaxPool2d-5             [-1, 32, 7, 7]               0
       BatchNorm2d-6             [-1, 32, 7, 7]              64
            Conv2d-7             [-1, 64, 7, 7]          18,496
         MaxPool2d-8             [-1, 64, 3, 3]               0
       BatchNorm2d-9             [-1, 64, 3, 3]             128
           Conv2d-10            [-1, 128, 3, 3]          73,856
        MaxPool2d-11            [-1, 128, 1, 1]               0
           Linear-12                  [-1, 512]          66,048
           Linear-13                  [-1, 256]         131,328
           Linear-14                   

In [15]:
opt = Adam(params=cnn_model.parameters(),lr=0.001)

In [16]:
from torch.utils.data import DataLoader, TensorDataset

In [17]:
train_loader = DataLoader(TensorDataset(x_train.float(),y_train),
                          batch_size=64,shuffle=True)

In [18]:
epochs=50
cnn_model=cnn_model.to("cuda")

for epoch in range(epochs):
  training_loss=0.0
  for batch,target in train_loader:
    batch = batch.to("cuda")
    target = target.to("cuda")

    opt.zero_grad()
    output = cnn_model(batch)
    loss = F.cross_entropy(output,target)
    loss.backward()
    opt.step()
    training_loss += loss.item()

if (epoch+1)%5 ==0:
  print(f"Training loss: {training_loss}")


Training loss: 3.930609177180515


In [20]:
preds_test = cnn_model(x_test.float().to("cuda")).detach().cpu().numpy()

In [21]:
pred = np.argmax(preds_test,axis = 1)

In [22]:
import pandas as pd

In [23]:
pd.crosstab(pred,y_test.numpy())

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,975,0,0,0,0,1,2,0,1,0
1,0,1134,0,0,7,0,1,11,0,3
2,2,0,1028,2,2,0,0,4,1,0
3,0,0,0,1007,0,8,0,1,1,0
4,0,0,1,0,963,0,0,0,0,3
5,0,0,0,1,0,881,2,0,0,4
6,1,0,0,0,1,1,953,0,0,1
7,1,1,3,0,0,0,0,1009,0,1
8,0,0,0,0,0,1,0,0,969,3
